In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree

# Test for GPU or CPU devices
print("Device: \n", tf.config.experimental.list_physical_devices())
print(tf.__version__)
print(tf.test.is_built_with_cuda())

# CLAHE Function for preprocessing
def clahe_function(img):
    clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(8,8))
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab_img)
    clahe_img = clahe.apply(l)
    updated_lab_img2 = cv2.merge((clahe_img, a, b))
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    return CLAHE_img

# Load images function
def load_images_from_folder(folder_path):
    images = []
    labels = []
    class_folders = os.listdir(folder_path)
    
    for label, class_folder in enumerate(class_folders):
        class_folder_path = os.path.join(folder_path, class_folder)
        for filename in os.listdir(class_folder_path):
            img_path = os.path.join(class_folder_path, filename)
            img = cv2.imread(img_path)
            img = clahe_function(img)
            img = cv2.resize(img, (224, 224))
            img = img / 255.0
            if img is not None:
                images.append(img)
                labels.append(label)
    return np.array(images), np.array(labels)

# Load datasets
train_folder = r"E:/sabbir/train"
test_folder = r"E:/sabbir/test"
valid_folder = r"E:/sabbir/valid"

X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)
X_valid, y_valid = load_images_from_folder(valid_folder)

# Merge all datasets
X_all = np.concatenate((X_train, X_test, X_valid), axis=0)
y_all = np.concatenate((y_train, y_test, y_valid), axis=0)

# Split the merged dataset
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_all, y_all, test_size=0.20, stratify=y_all, random_state=0)

# CNN Model
model = tf.keras.Sequential()
# 1st conv layer
model.add(tf.keras.layers.Conv2D(32, 3, input_shape=(224, 224, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
# 2nd conv layer
model.add(tf.keras.layers.Conv2D(64, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
# 3rd conv layer
model.add(tf.keras.layers.Conv2D(128, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
# 4th conv layer
model.add(tf.keras.layers.Conv2D(256, 3, padding="valid"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(2))
# Flatten Layer
model.add(tf.keras.layers.Flatten())
# Dense Layer 1
model.add(tf.keras.layers.Dense(8000))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1024))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, name='feature_denseee'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
# Dense Layer 2
model.add(tf.keras.layers.Dense(100))  # 100 Prominent Features are Extracted From This Layer
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dropout(0.5))
# Output Dense Layer
model.add(tf.keras.layers.Dense(4))
model.add(tf.keras.layers.Activation('softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='sparse_categorical_crossentropy', metrics=['acc'], optimizer=adam)
model.summary()


# Train the CNN model
history = model.fit(X_train_split, y_train_split, batch_size=32, epochs=100, verbose=1, validation_data=(X_test_split, y_test_split))

# Save and reload the model
model.save(r'E:/model2/save.h5')
model = tf.keras.models.load_model(r'E:/model2/save.h5')

# Create intermediate model for feature extraction
intermediate_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer('feature_denseee').output)
intermediate_layer_model.summary()

# Feature extraction
feature_engg_data = intermediate_layer_model.predict(X_train_split)
feature_engg_data = pd.DataFrame(feature_engg_data)

# Save and load the features
feature_engg_data.to_pickle(r'E:/model2/finalfeaturescovid.pkl')
features = pd.read_pickle(r'E:/model2/finalfeaturescovid.pkl')

# Normalize features
x = feature_engg_data.values
x = StandardScaler().fit_transform(x)

# Apply SVD for dimensionality reduction
svd = TruncatedSVD(n_components=100)
x_reduced = svd.fit_transform(x)

# Split the reduced features into Training & Testing Set
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(x_reduced, y_train_split, test_size=0.20, stratify=y_train_split, random_state=0)

# Ensure the labels are in integer format
def is_one_hot(encoded_labels):
    return len(encoded_labels.shape) > 1 and encoded_labels.shape[1] > 1

if is_one_hot(y_train_split):
    y_train_split = np.argmax(y_train_split, axis=1)
if is_one_hot(y_test_split):
    y_test_split = np.argmax(y_test_split, axis=1)

# Apply Machine Learning Algorithms for Classification
# Gaussian Naive Bayes
model_gnb = GaussianNB()
model_gnb.fit(X_train_split, y_train_split)
y_pred_gnb = model_gnb.predict_proba(X_test_split)

# Support Vector Machine
svclassifier = SVC(kernel='sigmoid', probability=True)
svclassifier.fit(X_train_split, y_train_split)
y_pred_svc = svclassifier.predict_proba(X_test_split)

# Gradient Boosting Machine
model_gbm = GradientBoostingClassifier(n_estimators=500, random_state=1)
model_gbm.fit(X_train_split, y_train_split)
y_pred_gbm = model_gbm.predict_proba(X_test_split)

# k-Nearest Neighbors
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train_split, y_train_split)
y_pred_knn = model_knn.predict_proba(X_test_split)

# Random Forest Classifier
RF = RandomForestClassifier(n_estimators=100, random_state=1)
RF.fit(X_train_split, y_train_split)
y_pred_rf = RF.predict_proba(X_test_split)

# Ensemble Model
voting_classifier = VotingClassifier(estimators=[('GNB', model_gnb), ('SVM', svclassifier), ('GBM', model_gbm), ('KNN', model_knn), ('RF', RF)], voting='soft')
voting_classifier.fit(X_train_split, y_train_split)
y_pred_vot = voting_classifier.predict_proba(X_test_split)

# Binarize the output labels
y_test_binarized = label_binarize(y_test_split, classes=[0, 1, 2, 3])

# Dictionary to store ROC AUC for each model
roc_auc = {}

# Compute ROC curves and AUCs
for model_name, y_pred_proba in {
    "GNB": y_pred_gnb,
    "SVM": y_pred_svc,
    "GBM": y_pred_gbm,
    "KNN": y_pred_knn,
    "RF": y_pred_rf,
    "Ensemble": y_pred_vot
}.items():
    fpr = dict()
    tpr = dict()
    roc_auc[model_name] = dict()
    
    for i in range(4):  # Assuming 4 classes
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
        roc_auc[model_name][i] = auc(fpr[i], tpr[i])

    plt.figure()
    for i in range(4):  # Plotting ROC curves for each class
        plt.plot(fpr[i], tpr[i], lw=2, label=f'Class {i} (area = {roc_auc[model_name][i]:.2f})')
    
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {model_name}')
    plt.legend(loc='lower right')
    plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

def generate_gradcam_heatmap(model, img_array, class_index):
    # Get the last convolutional layer
    last_conv_layer = model.get_layer('conv2d_7')  # Adjust to your last conv layer's name
    
    # Create a model that outputs the last conv layer and the model's output
    grad_model = tf.keras.models.Model(
        inputs=[model.input],
        outputs=[last_conv_layer.output, model.output]
    )
    
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]
    
    # Compute gradients of the loss w.r.t. the last conv layer output
    grads = tape.gradient(loss, conv_outputs)
    
    # Pool the gradients over all the axes
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # Get the values of the conv layer outputs and gradients
    conv_outputs_value = conv_outputs[0].numpy()
    pooled_grads_value = pooled_grads.numpy()
    
    # Apply the pooled gradients to the conv layer outputs
    for i in range(pooled_grads_value.shape[-1]):
        conv_outputs_value[:, :, i] *= pooled_grads_value[i]
    
    # Average the weighted conv layer outputs to get the heatmap
    heatmap = np.mean(conv_outputs_value, axis=-1)
    
    # Normalize the heatmap
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    return heatmap

def overlay_heatmap(heatmap, img, alpha=0.4):
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    return np.uint8(superimposed_img)

# Select 4 sample images and their respective target classes
sample_images = X_test[:4]  # Assuming X_test has at least 4 images
sample_images_exp = np.expand_dims(sample_images, axis=-1)  # Add a dimension for batch

# Display the original images and Grad-CAM heatmaps for 4 images
plt.figure(figsize=(12, 12))

for i in range(4):
    sample_image = sample_images[i]
    sample_image_exp = np.expand_dims(sample_image, axis=0)  # Expand to match model input shape
    class_index = np.argmax(model.predict(sample_image_exp))  # Class index for Grad-CAM

    # Generate Grad-CAM heatmap
    heatmap = generate_gradcam_heatmap(model, sample_image_exp, class_index)

    # Convert image to original format
    sample_image = sample_image * 255.0
    sample_image = sample_image.astype(np.uint8)

    # Overlay the heatmap on the image
    overlayed_img = overlay_heatmap(heatmap, sample_image)

    # Plot the original image and heatmap for each sample
    plt.subplot(4, 2, i * 2 + 1)
    plt.title(f'Original Image {i+1}')
    plt.imshow(sample_image)
    plt.axis('off')

    plt.subplot(4, 2, i * 2 + 2)
    plt.title(f'Grad-CAM Heatmap {i+1}')
    plt.imshow(overlayed_img)
    plt.axis('off')

plt.tight_layout()
plt.show()
